In [ ]:
!pip install -q transformers
!pip install -q sentencepiece
!pip install -q openpyxl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.5 MB/s eta 0:00:00


In [ ]:
import openpyxl
from transformers import MBart50Tokenizer, MBartForConditionalGeneration

def translate_text_mbart(texts, tokenizer, model, source_lang, target_lang):
    # Tokenize the input texts
    inputs = tokenizer.batch_encode_plus(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Translate the input texts
    translated = model.generate(inputs.input_ids, decoder_start_token_id=model.config.pad_token_id, max_length=512)

    # Decode the translated texts
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)

    return translated_texts

# Load the XLSX file
file_path = "dosyatest.xlsx"
workbook = openpyxl.load_workbook(file_path)

source_language = "tr"
target_language = "en"

# Initialize the model and tokenizer
model_name = "facebook/mbart-large-50-many-to-one-mmt"
tokenizer = MBart50Tokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# Translate the first column in each sheet
translated_texts = []
for sheet in workbook:
    column_values = [cell.value for cell in sheet['A'] if cell.value is not None]
    if not column_values:
        continue  # Skip translation if the column is empty

    # Translate 10 rows at a time
    batch_size = 10
    for i in range(0, len(column_values), batch_size):
        batch = column_values[i:i+batch_size]
        translated_column = translate_text_mbart(batch, tokenizer, model, source_language, target_language)
        translated_texts.extend(translated_column)

        # Update the translated text values in the workbook
        row_idx = i + 1
        for cell_value in translated_column:
            cell = sheet.cell(row=row_idx, column=1)
            if cell.coordinate in sheet.merged_cells:
                continue  # Skip merged cells
            cell.value = f"{cell_value} (Row {row_idx + 1})"
            row_idx += 1

# Save the translated XLSX file to a new filename
translated_file_path = "translated_dosya2.xlsx"
workbook.save(translated_file_path)
